In [3]:
def format_price(val):
    if val == 0:
        return '0'
    if val > 0.01:
        return f'{val:4.2f}'
    if val > 0.001:
        return f'{val:5.3f}'
    if val > 0.0001:
        return f'{val:6.4f}'
    if val > 0.00001:
        return f'{val:7.5f}'
    if val > 0.000001:
        return f'{val:8.6f}'
    return str(val)


import dateutil.parser


def make_token_card(token_id):
    token_data = tokens_ds[str(token_id)]
    title = token_data['info_title']
    description = token_data['info_description']
    mint_date = token_data['mint_iso_date'].replace('T', ' ').split()[0].replace('Z', '')
    mint_date = dateutil.parser.isoparse(mint_date).strftime("%B %d, %Y")
    corner_label = '<div class="ui right corner large label"><i class="cube icon"></i></div>'
    sold_count = tokens_by_sold_count[str(token_id)]
    available_count = token_data['available_count']
    total_sold_price = tokens_by_total_price[str(token_id)]
    available_price = token_data['available_prices_min']
    mime = token_data['artifact_mime']

    thumbnail_file = thumbs_dir / (token_id + '.jpeg')
    if not thumbnail_file.is_file():
        image_tag = (
            f'<div style="width: 256px; height: 256px; text-align: center; padding-top: 110px">'
            'click to view on hicetnunc.xyz</div>'
        )
    else:
        image_tag = (
            f'<img src="../token_thumbs/{token_id}.jpeg" '
            'style="height: 256px; width: 100%; object-fit: scale-down;">'
        )

    if mime == 'image/gif':
        corner_label = '<div class="ui right corner large label" style="text-align: right; padding-right: 5px; padding-top: 7px">GIF</div>'
    elif mime.startswith('image/svg'):
        corner_label = '<div class="ui right corner large label" style="text-align: right; padding-right: 5px; padding-top: 7px">SVG</div>'
    elif mime == 'application/pdf':
        corner_label = '<div class="ui right corner large label" style="text-align: right; padding-right: 5px; padding-top: 7px">PDF</div>'
    elif mime.startswith('video/'):
        corner_label = '<div class="ui right corner large label"><i class="play circle outline icon"></i></div>'
    elif mime.startswith('model/'):
        corner_label = '<div class="ui right corner large label"><i class="cube icon"></i></div>'
    elif mime.startswith('audio/'):
        corner_label = '<div class="ui right corner large label"><i class="file audio outline icon"></i></div>'
    else:
        corner_label = ''

    if total_sold_price > 0:
        avg_price = total_sold_price / sold_count
        price_text = f'Sold {sold_count} &times; {format_price(avg_price)}Ꜩ'
    elif sold_count:
        price_text = f'Sold {sold_count} for free'
    else:
        price_text = 'Nothing sold, not available for sale'

    if available_count:
        price_text += f',<br/>avail. at {format_price(available_price)}Ꜩ'

    if available_count:
        progress_color = 'green'
        progress_percent = int(sold_count / (sold_count + available_count) * 100)
    elif sold_count:
        progress_color = 'error'
        progress_percent = 100
    else:
        progress_color = 'warning'
        progress_percent = 100

    return f'''
    <div class="ui card">
      <div class="image">
        {corner_label}
        <a href="https://www.hicetnunc.xyz/objkt/{token_id}" target="_blank">
          {image_tag}
        </a>
      </div>
      <div class="content">
        <a href="https://www.hicetnunc.xyz/objkt/{token_id}" target="_blank" class="header">
            {title}
        </a>
        <div class="meta">
          <span class="date">{mint_date}</span>
        </div>
        <div class="description">
          {description}
        </div>
      </div>
      <div class="extra content">
         {price_text}
      </div>
      <div class="ui bottom attached {progress_color} progress progress-0" data-percent="{progress_percent}">
        <div class="bar"></div>
      </div>
    </div>
    '''


google_analytics = '''
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async src="https://www.googletagmanager.com/gtag/js?id=G-0K4ZXSKTS8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'G-0K4ZXSKTS8');
</script>
'''

from pathlib import Path

www_dir = Path('../../../hashquine.github.io/hicetnunc').resolve()
assert www_dir.is_dir()

thumbs_dir = www_dir / 'token_thumbs'

dataset_dir = Path('../../dataset').resolve()
assert dataset_dir.is_dir()

import json

addrs_ds = json.loads((dataset_dir / 'addrs.json').read_text('utf-8'))
tokens_ds = json.loads((dataset_dir / 'tokens.json').read_text('utf-8'))
sells_ds = json.loads((dataset_dir / 'sells.json').read_text('utf-8'))

from collections import Counter, defaultdict


ratings_specs = [{
    'min_date': '2021-03-01T00:00:00Z',
    'max_date': '2021-03-15T00:00:00Z',
    'dir_name': 'artworks-by-income-1',
    'tab_title': 'minted March 1 &ndash; 14',
}, {
    'min_date': '2021-03-15T00:00:00Z',
    'max_date': '2021-04-01T00:00:00Z',
    'dir_name': 'artworks-by-income-2',
    'tab_title': 'minted March 15 &ndash; 31',
}, {
    'min_date': '2021-04-01T00:00:00Z',
    'max_date': '2021-04-07T00:00:00Z',
    'dir_name': 'artworks-by-income-3',
    'tab_title': 'minted April 1 &ndash; 6',
}, {
    'min_date': '2021-04-07T00:00:00Z',
    'max_date': '2021-04-14T00:00:00Z',
    'dir_name': 'artworks-by-income-4',
    'tab_title': 'minted April 7 &ndash; 13',
}]

items_per_page = 100

for cur_rating_spec_no, rating_spec in enumerate(ratings_specs):
    rating_dir = www_dir / rating_spec['dir_name']
    rating_dir.mkdir(exist_ok=True)

    artists_by_income = Counter()
    artists_by_royalties = Counter()
    artists_by_spendings = Counter()
    tokens_by_total_price = Counter()
    tokens_by_sold_count = Counter()
    artist2tokens = defaultdict(set)

    for sell in sells_ds.values():
        token_mint_date = tokens_ds[sell['token_id']]['mint_iso_date']
        sell_date = sell['tr_iso_date']
        filter_date = token_mint_date

        if filter_date < rating_spec['min_date']:
            continue
        if filter_date >= rating_spec['max_date']:
            continue

        artists_by_royalties[sell['author']] += sell['total_royalties']

        if sell['by_author']:
            artists_by_income[sell['seller']] += sell['total_seller_income']
            tokens_by_total_price[sell['token_id']] += sell['price'] * sell['count']
            tokens_by_sold_count[sell['token_id']] += 1

        artists_by_spendings[sell['buyer']] += sell['price'] * sell['count']

        artist2tokens[sell['author']].add(sell['token_id'])


    for it in addrs_ds.values():
        if it['ban_status']:
            del artists_by_income[it['address']]
            del artists_by_royalties[it['address']]
            del artists_by_spendings[it['address']]

    for it in tokens_ds.values():
        if it['ban_status']:
            del tokens_by_total_price[it['token_id']]

    pages_addresses = ['index.html'] + [f'page_{i}.html' for i in range(1, 10)]

    for cur_page_no, cur_page_addr in enumerate(pages_addresses):

        html = ''
        html += '<head>'
        html += '<title>Artworks rating (hicetnunc.xyz)</title>'
        html += '''
        <meta name="twitter:card" content="summary_large_image">
        <meta name="twitter:creator" content="@pallada92">
        <meta name="twitter:title" content="Top-1000 hicetnunc.xyz artworks by income">
        <meta name="twitter:description" content="Based on data from Tezos blockchain and IPFS">
        <meta name="twitter:image" content="https://hashquine.github.io/hicetnunc/artworks-by-income-2/twitter_preview.png">
        '''
        html += '<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/github-fork-ribbon-css/0.2.3/gh-fork-ribbon.min.css" />'
        html += google_analytics
        html += '</head>'
        html += '<body style="margin: 0px auto; padding: 0px 20px; max-width: 1200px">'

        html += '<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/semantic-ui@2.4.2/dist/semantic.min.css">'
        html += (
            '<script src="https://code.jquery.com/jquery-3.1.1.min.js" '
            'integrity="sha256-hVVnYaiADRTO2PzUGmuLJr8BLUSjGIZsDYGmIJLv2b8=" crossorigin="anonymous"></script>'
        )
        html += '<script src="https://cdn.jsdelivr.net/npm/semantic-ui@2.4.2/dist/semantic.min.js"></script>'

        fork_url = 'https://github.com/hashquine/hicetnunc-dataset/blob/master/jupyter/output_html/artworks_rating_by_income.ipynb'
        html += f'<a class="github-fork-ribbon" href="{fork_url}" data-ribbon="Fork me on GitHub" title="Fork me on GitHub">Fork me on GitHub</a>'

        html += '''<div class="ui top fixed menu">
          <a class="item" href="../artists-by-income-4/index.html">Artists by income</a>
          <a class="item active" href="../artworks-by-income-4/index.html">Artworks by income</a>
        </div><div style="margin-top: 100px"></div>'''

        dataset_files = [{
            'fname': 'addrs',
            'formats': ['csv', 'json'],
            'url': 'https://raw.githubusercontent.com/hashquine/hicetnunc-dataset/master/dataset/addrs',
            'fields_descr_url': 'https://github.com/hashquine/hicetnunc-dataset/#addrsjson-and-addrscsv--of-all-hicetnunc-users',
            'description': 'All hicetnunc users with tzkt.io metadata and aggregated prices indicators',
        }, {
            'fname': 'tokens',
            'formats': ['csv', 'json'],
            'url': 'https://raw.githubusercontent.com/hashquine/hicetnunc-dataset/master/dataset/tokens',
            'fields_descr_url': 'https://github.com/hashquine/hicetnunc-dataset/#tokensjson-and-tokenscsv--of-all-nfts-tokens',
            'description': 'All hicetnunc objkts and aggregated prices indicators',
        }, {
            'fname': 'sells',
            'formats': ['csv', 'json'],
            'url': 'https://raw.githubusercontent.com/hashquine/hicetnunc-dataset/master/dataset/sells',
            'fields_descr_url': 'https://github.com/hashquine/hicetnunc-dataset/#sellsjson-and-sellscsv--of-all-purchases-via-official-hicetnunc-swaps',
            'description': 'Every sell event on hicetnunc website',
        }, {
            'fname': 'swaps',
            'formats': ['csv', 'json'],
            'url': 'https://raw.githubusercontent.com/hashquine/hicetnunc-dataset/master/dataset/swaps',
            'fields_descr_url': 'https://github.com/hashquine/hicetnunc-dataset/#swapsjson-and-swapscsv--all-official-hicetnunc-swaps-ever-created',
            'description': 'All hicetnunc swaps that ever existed',
        }, {
            'fname': 'transfers',
            'formats': ['csv', 'json'],
            'url': 'https://raw.githubusercontent.com/hashquine/hicetnunc-dataset/master/dataset/transfers',
            'fields_descr_url': 'https://github.com/hashquine/hicetnunc-dataset/#transfersjson-and-transferscsv--all-token-transfers',
            'description': 'All hicetnunc token transfers including external services like quipuswap',
        }, {
            'fname': 'token_thumbs',
            'formats': ['images'],
            'url': 'https://github.com/hashquine/hashquine.github.io/tree/master/hicetnunc/token_thumbs',
            'description': 'Previews for some hicetnunc tokens exluding interactive ones',
        }, {
            'fname': 'user_logos',
            'formats': ['images'],
            'url': 'https://github.com/hashquine/hashquine.github.io/tree/master/hicetnunc/user_logos',
            'description': 'Logos of some users taken from tzkt.io',
        }]

        html += '<table class="ui celled striped table" style="margin-bottom: 50px">'
        github_url = 'https://github.com/hashquine/hicetnunc-dataset'
        github_link = f'<a href="{github_url}" target="_blank">{github_url}</a>'
        html += f'<thead><tr><th colspan="4">Hicetnunc open dataset and parsers: {github_link}</th></tr></thead>'
        html += '<tbody>'

        for dataset_file in dataset_files:
            html += '<tr>'
            html += '<td class="collapsing right aligned">'
            fname = dataset_file['fname']
            url = dataset_file['url']
            if dataset_file['formats'] == ['images']:
                icon = f'<i class="folder open outline icon"></i>'
                html += f'<a href="{url}" target="_blank">{fname}</a>'
                size = ''
                html += f'</td><td class="collapsing"></td>'
            else:
                icon = f'<i class="table icon"></i>'
                html += f'<a href="{url}.csv" target="_blank">{fname}.csv</a><br/>'
                html += f'<a href="{url}.json" target="_blank">{fname}.json</a>'
                csv_size = (dataset_dir / (fname + '.csv')).stat().st_size / 1024 / 1024
                json_size = (dataset_dir / (fname + '.json')).stat().st_size / 1024 / 1024
                size = f'{csv_size:5.2f} Mb<br/>{json_size:5.2f} Mb'
                fields_descr_url = dataset_file['fields_descr_url']
                html += f'</td><td class="collapsing" align="center"><a href="{fields_descr_url}" target="_blank">fields<br/>description</a></td>'
            html += '</td>'
            html += '<td class="">' + icon + ' ' + dataset_file['description'] + '</td>'
            html += f'<td class="right aligned collapsing">{size}</td>'

            html += '</tr>'

        html += '</tbody></table>'

        html += (
            '<h1>Unofficial <a href="https://hicetnunc.xyz/" target="_blank">hicetnunc.xyz</a> '
            'rating of highest grossing artworks</h1>'
        )

        html += (
            '<p align="left">'
            'Project by&nbsp; <a href="https://twitter.com/HashQuine" target="_blank">'
            '<i class="twitter icon"></i>HashQuine</a>. '
            'Based on data from Tezos blockchain (by <a href="https://tzstats.com/docs/api" target="_blank">TzStats API</a>), '
            'IPFS and <a href="http://tzkt.io/">tzkt.io</a> accounts metadata API. '
            'No requests to hicetnunc.xyz were sent during making this rating.'
            '</p>'
        )

        html += '<p align="right">Last updated April 7, 2021 20:00 UTC</p>'

        html += '<div class="ui four item pointing menu">'
        for other_rating_spec_no, other_rating_spec in enumerate(ratings_specs):
            active = ' active' if other_rating_spec_no == cur_rating_spec_no else ''
            other_rating_spec_title = other_rating_spec['tab_title']
            other_rating_spec_url = '../' + other_rating_spec['dir_name'] + '/index.html'
            html += f'<a class="item{active}" style="" href="./{other_rating_spec_url}">{other_rating_spec_title}</a>'
        html += '</div>'

        html += '<div class="ui ten item pointing menu">'
        for other_page_no, other_page_addr in enumerate(pages_addresses):
            active = ' active' if other_page_no == cur_page_no else ''
            other_range = f'{other_page_no * items_per_page + 1}&ndash;{other_page_no * items_per_page + items_per_page}'
            html += f'<a class="item{active}" style="" href="./{other_page_addr}">{other_range}</a>'
        html += '</div>'

        html += '''
        <div class="ui icon message" style="padding-bottom: 0; margin-bottom: 0px">
          <i class="exclamation triangle icon"></i>
          <div class="content">
            <div class="header">
              Word of caution:
            </div>
            <ul>
                <li>
                    We haven't validated, that the artists accounts presented here are really owned by the artists themselves.
                    Please double check before purchasing.
                </li>
                <li>
                    These metrics can be easily cheated by artists creating transactions between fake accounts.
                    Do not rely on this data alone!
                </li>
            </ul>
          </div>
        </div>
        '''

        html += '<div class="ui divider"></div>'
        html += '<div class="ui four doubling cards" style="flex-grow: 1; margin-right: 15px">'

        for token_no, (token_id, token_income) in enumerate(tokens_by_total_price.most_common()):
            if not (items_per_page > token_no - cur_page_no * items_per_page >= 0):
                continue

            html += make_token_card(token_id)

        html += '</div>'

        html += f'<script>$(".progress-0").progress(); </script>'

        html += '<div class="ui ten item menu">'
        for other_page_no, other_page_addr in enumerate(pages_addresses):
            active = ' active' if other_page_no == cur_page_no else ''
            other_range = f'{other_page_no * items_per_page + 1}&ndash;{other_page_no * items_per_page + items_per_page}'
            html += f'<a class="item{active}" style="" href="./{other_page_addr}">{other_range}</a>'
        html += '</div>'
        html += '<div style="height: 50px"></div>'

        html += '</body>'

        (rating_dir / cur_page_addr).write_text(html, 'utf-8')